In [10]:
%pip install langchain_openai langchain_community langchain pymysql chromadb -q
%pip install -U langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install sqlite3
%pip install pandas
%pip install sqlalchemy

In [78]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
# from langchain import LangSmithClient

os.environ["OPENAI_API_KEY"] = "sk-proj-SQ4pUgginaQryU09DBN6T3BlbkFJFXU3E4Nl4yy6W2wCB2Ku"

os.environ["LANGCHAIN_TRACING_V2"]= "true"

os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

os.environ["LANGCHAIN_API_KEY"]="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"

db_uri = r'sqlite:///C:/Users/digan/OneDrive/Documents/sqlite'

# sqlite:///C:/Users/digan/AppData/Roaming/DBeaverData/workspace6/.metadata/sample-database-sqlite-1/Chinook.db

db_name = "classicmodels"

# {db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_uri}@{db_name}")
db= SQLDatabase.from_uri(db_uri)

# db = SQLDatabase.from_uri(
#     db_uri,
#     sample_rows_in_table_info=1,  # Include sample rows in table info
#     include_tables=['customers', 'orders'],  # Include specific tables
#     custom_table_info={'customers': "customers"}  # Custom table info
# )

print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)


sqlite
['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']

CREATE TABLE customers (
	"customerNumber" INTEGER, 
	"customerName" VARCHAR(50) NOT NULL, 
	"contactLastName" VARCHAR(50) NOT NULL, 
	"contactFirstName" VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	"addressLine1" VARCHAR(50) NOT NULL, 
	"addressLine2" VARCHAR(50) DEFAULT NULL, 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50) DEFAULT NULL, 
	"postalCode" VARCHAR(15) DEFAULT NULL, 
	country VARCHAR(50) NOT NULL, 
	"salesRepEmployeeNumber" INTEGER DEFAULT NULL, 
	"creditLimit" DECIMAL(10, 2) DEFAULT NULL, 
	PRIMARY KEY ("customerNumber"), 
	FOREIGN KEY("salesRepEmployeeNumber") REFERENCES employees ("employeeNumber")
)

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale

c:\Users\digan\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\utilities\sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.SMALLINT'> with reflected arguments ['6']; using no arguments.
  self._metadata.reflect(


In [64]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "what is price of `1968 Ford Mustang`"})
# "what is price of `1968 Ford Mustang`"
print(query)


SELECT "productName", "buyPrice" 
FROM products 
WHERE "productName" = '1968 Ford Mustang'
LIMIT 1;


In [65]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

"[('1968 Ford Mustang', 95.34)]"

In [66]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "How many orders are there"})


'There are a total of 326 orders.'

In [67]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
]


In [68]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input","top_k"],
    input_variables=["input"],
)
print(few_shot_prompt.format(input1="How many products are there?"))


Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;


In [69]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))


Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;


In [70]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info"))
generate_query = create_sql_query_chain(llm, db,final_prompt)
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many customers with credit limit more than 50000"})


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: How many products are there?


'There are 85 customers with a credit limit greater than 50000.'

In [71]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("classicmodels.csv")
    table_docs = []

    # "customerNumber","customerName","contactLastName","contactFirstName","phone","addressLine1","addressLine2",
    # "city","state","postalCode","country","salesRepEmployeeNumber","creditLimit","orderNumber","orderDate","requiredDate",
    # "shippedDate","status","comments","customerNumber"

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + (
        f"Customer Number: {row['customerNumber']}\n"
        f"Customer Name: {row['customerName']}\n"
        f"Contact Last Name: {row['contactLastName']}\n"
        f"Contact First Name: {row['contactFirstName']}\n"
        f"Phone: {row['phone']}\n"
        f"Address Line 1: {row['addressLine1']}\n"
        f"Address Line 2: {row['addressLine2']}\n"
        f"City: {row['city']}\n"
        f"State: {row['state']}\n"
        f"Postal Code: {row['postalCode']}\n"
        f"Country: {row['country']}\n"
        f"Sales Rep Employee Number: {row['salesRepEmployeeNumber']}\n"
        f"Credit Limit: {row['creditLimit']}\n"
        f"Order Number: {row['orderNumber']}\n"
        f"Order Date: {row['orderDate']}\n"
        f"Required Date: {row['requiredDate']}\n"
        f"Shipped Date: {row['shippedDate']}\n"
        f"Status: {row['status']}\n"
        f"Comments: {row['comments']}\n\n"
        )

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)


Customer Number: 103
Customer Name: Atelier graphique
Contact Last Name: Schmitt
Contact First Name: Carine 
Phone: 40.32.2555
Address Line 1: 54, rue Royale
Address Line 2: nan
City: Nantes
State: nan
Postal Code: 44000
Country: France
Sales Rep Employee Number: 1370
Credit Limit: 21000
Order Number: 10100
Order Date: 2003-01-06
Required Date: 2003-01-13
Shipped Date: 2003-01-10
Status: Shipped
Comments: nan

Customer Number: 103
Customer Name: Atelier graphique
Contact Last Name: Schmitt
Contact First Name: Carine 
Phone: 40.32.2555
Address Line 1: 54, rue Royale
Address Line 2: nan
City: Nantes
State: nan
Postal Code: 44000
Country: France
Sales Rep Employee Number: 1370
Credit Limit: 21000
Order Number: 10101
Order Date: 2003-01-09
Required Date: 2003-01-18
Shipped Date: 2003-01-11
Status: Shipped
Comments: Check on availability.

Customer Number: 103
Customer Name: Atelier graphique
Contact Last Name: Schmitt
Contact First Name: Carine 
Phone: 40.32.2555
Address Line 1: 54, rue Ro

In [72]:
table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

# {table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me details of customer and their order count"})
tables

[Table(name='Customers'), Table(name='Orders')]

In [73]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me details of customer and their order count"})

['Customers', 'Orders']

In [79]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many customers with orders count more than 5"})


ValueError: table_names {'Orders', 'Customers'} not found in database

In [49]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

In [50]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: How many products are there?


In [51]:
generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)

In [75]:
question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})
# There are 2 customers with an order count of more than 5.
  

ValueError: table_names {'Orders', 'Customers'} not found in database

In [76]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages})
response
# The names of the customers with more than 5 orders are Mini Gifts Distributors Ltd. and Euro+ Shopping Channel.


ValueError: table_names {'Orders', 'Employees', 'Customers'} not found in database